# Imports & Configuration

In [18]:
!pip install wordsegment
!pip install num2words
!pip install skorch
import os
import string
import nltk
import torch
import re
import spacy
import time
import torch
import torch.nn as nn
import random
import numpy as np
import seaborn as sns
import pandas as pd
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import wordsegment
import sklearn
from google.colab import drive
from imblearn.over_sampling import SMOTE
from skorch.callbacks import Callback
from imblearn.over_sampling import ADASYN
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.optim import SGD
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from skorch import NeuralNetClassifier
from tqdm import tqdm_notebook
from skorch.callbacks import EpochScoring
from num2words import num2words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import KeyedVectors
from scipy.stats import randint
from sklearn.model_selection import ParameterGrid
from gensim.models import FastText
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from skorch.callbacks import EarlyStopping
from torch.utils.data import (TensorDataset, DataLoader, RandomSampler, SequentialSampler)
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, precision_recall_fscore_support
import seaborn as sns
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

%matplotlib inline

nltk.download("all")
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    |   Package bcp47 is already up-to-dat

In [6]:
# setting the seed for reproducability
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

set_seed(42)

In [7]:
# switch to gpu if available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla V100-SXM2-16GB


In [8]:
# connect to the drive, remove if used on local device
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preparation

## Loading Data

In [9]:
# Loading communication data, change this path when using this code on your local device
df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/communication_data.xlsm')

## Data Processing

In [10]:
# concatenating and reorganizing data
df = df.groupby('NegotiationID').agg({
    'NegoOutcome': 'first',
    'Content': lambda x: ' '.join(x)
}).reset_index()

texts = df['Content'].tolist()
labels = [0 if negoOutcome == "FinalAccept" else 1 for negoOutcome in df['NegoOutcome'].tolist()]

In [11]:
# Defining BERT Data Preparation

max_length = 510

class TextClassificationDataset(Dataset):

    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts # At this point, text input length can be cut to only use characters
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # truncation_strategy defines if the input is truncated from the front or the back, options: "only_second" / "only_first"
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True, truncation_strategy='only_second')
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

# Model definition

## Parameters

In [13]:
# Set up parameters, will be adjusted in search space
bert_model_name = 'bert-base-uncased'
num_classes = 2
batch_size = 16
num_epochs = 6
learning_rate = 3e-5

# Calculate weights inversely proportional to class frequencies
positive_samples = 105
negative_samples = 518
total_samples = positive_samples + negative_samples

weight_negative = total_samples / (2 * negative_samples)
weight_positive = total_samples / (2 * positive_samples)
class_weights = torch.tensor([weight_negative, weight_positive], dtype=torch.float)
class_weights.to(device)

tensor([0.6014, 2.9667], device='cuda:0')

## Model and functions

In [14]:
# Define BERT Model
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [15]:
# Define train loop
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss(weight = torch.tensor([weight_negative, weight_positive]).to(device))(outputs, labels)
        loss.to(device)
        loss.backward()
        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
    average_loss = total_loss / len(data_loader)
    return average_loss


In [16]:
# Define eval loop
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = nn.CrossEntropyLoss(weight=torch.tensor([weight_negative, weight_positive]).to(device))(outputs, labels)
            total_loss += loss.item()

            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())

    accuracy = accuracy_score(actual_labels, predictions)
    average_loss = total_loss / len(data_loader)
    precision, recall, fscore, support = precision_recall_fscore_support(actual_labels, predictions)

    print(classification_report(actual_labels, predictions))
    return average_loss, accuracy, precision[1], recall[1], fscore[1], fscore[0]

# Model Execution

## Cross Fold Validation

In [17]:
# Define manual Cross Validation Loop
class KStratifiedCrossValidation:
    def __init__(self, k, batch_size, epochs, learning_rate):
        self.k = k
        self.batch_size = batch_size
        self.epochs = epochs
        self.learning_rate = learning_rate

    def k_stratified_cross_validation(self):

        # overall metrics
        fold_accuracies = []
        fold_precisions = []
        fold_recalls = []
        fold_fscores_1 = []
        fold_fscores_0 = []

        fold_training_losses = []  # training losses for each fold
        fold_validation_losses = []  # validation losses for each fold

        kf = StratifiedKFold(n_splits=self.k, shuffle=True, random_state=42)
        print(f"Performing Stratified-k-Fold-Crossvalidation with:\n  Epochs: {self.epochs}\n  Batch Size: {self.batch_size}\n  Number of Folds: {self.k}")

        # define tokenizer
        tokenizer = BertTokenizer.from_pretrained(bert_model_name)

        best_fold_accuracy = 0.0
        best_fold_fscore_1 = 0.0
        best_fold_index = 0
        best_model = None

        for fold, (train_index, test_index) in enumerate(kf.split(texts, labels)):
            print(f"Fold {fold + 1}/{self.k}")

            # Splitting data into folds
            fold_train_inputs, fold_validation_inputs = [texts[i] for i in train_index], [texts[i] for i in test_index]
            fold_train_labels, fold_validation_labels = [labels[i] for i in train_index], [labels[i] for i in test_index]

            # Encoding data and Loading fold data into PyTorch DataLoader
            train_dataset = TextClassificationDataset(fold_train_inputs, fold_train_labels, tokenizer, max_length)
            val_dataset = TextClassificationDataset(fold_validation_inputs, fold_validation_labels, tokenizer, max_length)
            fold_training_dataloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
            fold_validation_dataloader = DataLoader(val_dataset, batch_size=self.batch_size)

            # initialize model, optimizer and scheduler
            model = BERTClassifier(bert_model_name, num_classes).to(device)
            optimizer = AdamW(model.parameters(), lr=self.learning_rate)
            total_steps = len(fold_training_dataloader) * self.epochs
            scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

            # perform training and print results for the fold
            fold_accuracy_values = []
            fold_precision_values = []
            fold_recall_values = []
            fold_fscore_1_values = []
            fold_fscore_0_values = []

            fold_training_losses.append([])  # Initialize empty list for training losses
            fold_validation_losses.append([])

            print(f"Training and evaluating Fold number {fold + 1}/{self.k}")
            for epoch in range(self.epochs):
              print(f"Epoch {epoch + 1}/{self.epochs}")
              train_loss = train(model, fold_training_dataloader, optimizer, scheduler, device)
              fold_training_losses[fold].append(train_loss)  # Store training loss
              val_loss, accuracy, precision, recall, fscore_1, fscore_0 = evaluate(model, fold_validation_dataloader, device)
              fold_validation_losses[fold].append(val_loss)  # Store val loss
              print(f"Validation FScore_1: {fscore_1:.4f}")
              print(f"Validation FScore_0: {fscore_0:.4f}")
              print(f"Validation Precision_1: {precision:.4f}")
              print(f"Validation Recall_1: {recall:.4f}")
              print(f"Validation Accuracy: {accuracy:.4f}")

              # store report values for each epoch in the fold
              fold_accuracy_values.append(accuracy)
              fold_precision_values.append(precision)
              fold_recall_values.append(recall)
              fold_fscore_1_values.append(fscore_1)
              fold_fscore_0_values.append(fscore_0)

            # calculate and Store average report values for the fold
            mean_fold_accuracy = np.mean(fold_accuracy_values)
            mean_fold_precision = np.mean(fold_precision_values)
            mean_fold_recall = np.mean(fold_recall_values)
            mean_fold_fscore_1 = np.mean(fold_fscore_1_values)
            mean_fold_fscore_0 = np.mean(fold_fscore_0_values)

            fold_accuracies.append(mean_fold_accuracy)
            fold_precisions.append(mean_fold_precision)
            fold_recalls.append(mean_fold_recall)
            fold_fscores_1.append(mean_fold_fscore_1)
            fold_fscores_0.append(mean_fold_fscore_0)

            # update benchmark
            if mean_fold_fscore_1 > best_fold_fscore_1:
              best_fold_fscore_1 = mean_fold_fscore_1
              best_fold_index = fold
              best_model=model

        training_loss_best_model = fold_training_losses[best_fold_index]
        validation_loss_best_model = fold_validation_losses[best_fold_index]

        # print final results
        print(f"Cross Fold Validation Complete! Computing the report ...:")

        # Print and store overall results across all folds
        mean_accuracy = np.mean(fold_accuracies)
        mean_precision = np.mean(fold_precisions)
        mean_recall = np.mean(fold_recalls)
        mean_fscore_0 = np.mean(fold_fscores_0)
        mean_fscore_1 = np.mean(fold_fscores_1)

        print("Cross Validation Averaged Results overall:")
        print(f"Averaged accuracy over {self.k} Folds: {mean_accuracy:.4f}")
        print(f"Averaged precision over {self.k} Folds: {mean_precision:.4f}")
        print(f"Averaged recall over {self.k} Folds: {mean_recall:.4f}")
        print(f"Averaged fscore_1 over {self.k} Folds: {mean_fscore_1:.4f}")
        print(f"Averaged accuracy over {self.k} Folds: {mean_accuracy:.4f}")

        report = [mean_fscore_1, mean_fscore_0, mean_precision, mean_recall, mean_accuracy, fold_fscores_1]

        return report, best_model, training_loss_best_model, validation_loss_best_model

##  Optimization Loop

In [19]:
# Define manual Hyperparameter Optimization (Grid Search) with integrated cross fold validation

class HyperparameterOptimizer:
    def __init__(self, model_class, param_grid, k):
        self.model_class = model_class
        self.param_grid = param_grid
        self.k=k

    def evaluate_model(self, params):
        k_strat_cv = KStratifiedCrossValidation(self.k, params['batch_size'], params['epochs'], params['learning_rate'])
        report, best_cv_model, trainig_loss_best_cv_model, validation_loss_best_cv_model = k_strat_cv.k_stratified_cross_validation()

        return report, best_cv_model, trainig_loss_best_cv_model, validation_loss_best_cv_model

    def run_grid_search(self):
        best_params = None
        best_report = [0,0,0,0,0,0]
        best_overall_model=None
        training_loss_best_overall_model = None
        validation_loss_best_overall_model = None

        for params in ParameterGrid(self.param_grid):
            print(f"\nEvaluating hyperparameters: {params}")

            report, best_cv_model, training_loss_best_cv_model, validation_loss_best_cv_model = self.evaluate_model(params)

            if report[0] > best_report[0]:
                best_report = report
                best_params = params
                best_overall_model = best_cv_model
                training_loss_best_overall_model = training_loss_best_cv_model
                validation_loss_best_overall_model = validation_loss_best_cv_model

        print("\nGrid searched for the hyperparameter configuration in terms of fscore_1:")
        print("\nGrid search complete. Best hyperparameters:")
        print(best_params)
        print(f"Average FScore_1 over all folds of this configuration: {report[0]:.4f}")
        print(f"Average FScore_0 over all folds of this configuration: {report[1]:.4f}")
        print(f"Average Precision_1 over all folds of this configuration: {report[2]:.4f}")
        print(f"Average Recall_1 over all folds of this configuration: {report[3]:.4f}")
        print(f"Average Accuracy over all folds of this configuration: {report[4]:.4f}")

        standard_deviation_fscore_1 = np.std(report[5])
        print(standard_deviation_fscore_1)
        variation_coefficient_fscore_1 = (report[0]/standard_deviation_fscore_1)
        print(f"Standard Deviation of the fscore_1 for each fold of the best configuration: {standard_deviation_fscore_1:.4f}")
        print(f"Variance of the fscore_1 for each fold of the best configuration: {variation_coefficient_fscore_1:.4f}")

        print("Cross Validation Averaged Results for each fold of the best configuration:")
        print(report[5])

        return best_params, best_overall_model, training_loss_best_overall_model, validation_loss_best_overall_model

# Search Space, Example Usage, define at will:
param_grid = {
    'batch_size': [16, 32],
    'epochs': [4],
    'learning_rate': [3e-5],
}

# Define optimizer and set number of folds k
optimizer = HyperparameterOptimizer(BERTClassifier, param_grid, k=10)


## Execution and Plotting

In [19]:
# Execute the Hyperoptimization Loop with Cross fold validation and save the results
best_params, best_overall_model, training_loss_best_overall_model, validation_loss_best_overall_model = optimizer.run_grid_search()

In [ ]:
# Save the best model from the best fold
torch.save(best_overall_model, '/content/drive/MyDrive/Colab Notebooks/YOUR_MODEL_NAME.pth')

In [ ]:
# Plotting the loss chart
epochs = np.arange(1, len(training_loss_best_overall_model) + 1)

plt.figure(figsize=(10, 6))

# Plot training loss
plt.plot(epochs, training_loss_best_overall_model, label='Training Loss', marker='o')

# Plot validation loss
plt.plot(epochs, validation_loss_best_overall_model, label='Validation Loss', linestyle='--', marker='o')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Losses')
plt.legend()
plt.show()